In [16]:
import fastf1.events
from pptx import Presentation
from pptx.util import Pt
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_CONNECTOR
from pptx.enum.shapes import MSO_SHAPE
from pptx.enum.text import MSO_ANCHOR
from pptx.enum.text import PP_ALIGN
from pptx.oxml.xmlchemy import OxmlElement

import fastf1
import fastf1.plotting

import pandas as pd
import numpy as np
import os  
from datetime import timedelta
import csv
from fastf1.ergast import Ergast

ergast = Ergast()

year = 2025
race_number = 1

event = fastf1.get_event(year, race_number)

try:
    sprint_test = event.get_sprint()
    session_fp1= fastf1.get_session(year, race_number, 'FP1')
    session_fp1.load(weather=True)

    free_practice_session = [session_fp1]
except:
    session_fp1= fastf1.get_session(year, race_number, 'FP1')
    session_fp1.load(weather=True)

    session_fp2= fastf1.get_session(year, race_number, 'FP2')
    session_fp2.load(weather=True)

    session_fp3= fastf1.get_session(year, race_number, 'FP3')
    session_fp3.load(weather=True)

    free_practice_session = [session_fp1, session_fp2, session_fp3]

teams = fastf1.plotting.list_team_names(session_fp1)
event_name = session_fp1.event.EventName

core           INFO 	Loading data for Australian Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '7', '10', '12', '14', '16', '18', '22', '23', '27', '30', '31', '44', '55', '63', '81', '87']
core           INFO 	Loading data for Australian Grand Prix - Practice 2 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for t

In [17]:
def SubElement(parent, tagname, **kwargs):
        element = OxmlElement(tagname)
        element.attrib.update(kwargs)
        parent.append(element)
        return element

def _set_shape_transparency(shape, alpha):
    """ Set the transparency (alpha) of a shape"""
    ts = shape.fill._xPr.solidFill
    sF = ts.get_or_change_to_srgbClr()
    sE = SubElement(sF, 'a:alpha', val=str(alpha))
    
def Hex_RGB(ip):
    return tuple(int(ip[i+1:i+3],16) for i in (0, 2, 4))

In [ ]:
practice_track_status = {}
figures_path = '/home/kurios/Documents/f1_analysis/reports/'

for practice_session in free_practice_session:
    prs = Presentation()

    laps = practice_session.laps
    stints = laps[["Driver", "Stint", "Compound", "LapNumber", "LapTime"]]
    stints = stints.groupby(["Driver", "Stint", "Compound"]).agg(
        LapCount=('LapNumber', 'count'),
        FirstLap=('LapNumber', 'min'),
        LastLap=('LapNumber', 'max')
    ).reset_index()

    air_temp_range = str((min(practice_session.weather_data['AirTemp']), max(practice_session.weather_data['AirTemp']))) + ' °C'
    track_temp_range = str((min(practice_session.weather_data['TrackTemp']), max(practice_session.weather_data['TrackTemp']))) + ' °C'
    humidity_range = str((min(practice_session.weather_data['Humidity']), (max(practice_session.weather_data['Humidity'])))) + ' %'

    teams = fastf1.plotting.list_team_names(practice_session)
    
    for idx, team in enumerate(teams):
        team_drivers = fastf1.plotting.get_driver_abbreviations_by_team(team, session=practice_session)

        driver_info_1 = practice_session.get_driver(team_drivers[0])
        driver_info_2 = practice_session.get_driver(team_drivers[1])
        
        #DRIVER 1 DATA
        driver_1 = driver_info_1['DriverNumber']
        driver_1_name = driver_info_1['FullName']

        epsilon = 1e-6
        practice_session.car_data[driver_1]['is_driving'] = (practice_session.car_data[driver_1]['Speed'] > epsilon) & (practice_session.car_data[driver_1]['nGear'] > 0)
        practice_session.car_data[driver_1][practice_session.car_data[driver_1]['is_driving'] == True]
        practice_session.car_data[driver_1]['block_start'] = practice_session.car_data[driver_1]['is_driving'] & (~practice_session.car_data[driver_1]['is_driving'].shift(1, fill_value=False))
        practice_session.car_data[driver_1]['block_id'] = practice_session.car_data[driver_1]['block_start'].cumsum()
        df_driving_only = practice_session.car_data[driver_1][practice_session.car_data[driver_1]['is_driving']].copy()

        if not df_driving_only.empty:
            fastest_lap = practice_session.laps.pick_drivers(driver_1).pick_fastest()
            driver_1 = driver_info_1["Abbreviation"]

            driving_block_durations = df_driving_only[df_driving_only['block_id'] > 0].groupby('block_id')['SessionTime'].apply(lambda x: x.iloc[-1] - x.iloc[0])
            min_block_duration = pd.Timedelta(minutes=1)
            filtered_driving_block_durations = driving_block_durations[driving_block_durations >= min_block_duration]
            
            total_run_time_driver_1 = str(filtered_driving_block_durations.sum())[10:-3]
            try:
                fastest_lap_driver_1 = str(fastest_lap['LapTime'])[10:-3]
            except:
                fastest_lap_driver_1 = 'No Data'

            driver_stints_driver_1 = stints[stints['Driver'] == driver_1]

            data_driver_1 = []
            try:
                for index, row in driver_stints_driver_1.iterrows():
                    driver_lap = practice_session.laps.pick_drivers(driver_1).pick_laps(range(int(row['FirstLap']), int(row['LastLap'])+1)).pick_quicklaps()
                    number_lap_driver_1 = int(driver_lap['LapTime'].count())
                    driver_avg_lap_driver_1 = str(timedelta(seconds = driver_lap['LapTime'].dt.total_seconds().median()))[3:-3]
                    compound_driver_1 = stints['Compound'][index]
                    data_driver_1.append(compound_driver_1)
                    data_driver_1.append(number_lap_driver_1)
                    data_driver_1.append(driver_avg_lap_driver_1)
            except:
                driver_avg_lap_driver_1 = 'No Average Lap Time Found'
                compound_driver_1 = 'No Compound Found'
                number_lap_driver_1 = 'No Lap completed'
            data_driver_1_len = int(len(data_driver_1)/3)

        #DRIVER 2 DATA    
        driver_2 = driver_info_2['DriverNumber']
        driver_2_name = driver_info_2['FullName']

        epsilon = 1e-6
        practice_session.car_data[driver_2]['is_driving'] = (practice_session.car_data[driver_2]['Speed'] > epsilon) & (practice_session.car_data[driver_2]['nGear'] > 0)
        practice_session.car_data[driver_2][practice_session.car_data[driver_2]['is_driving'] == True]
        practice_session.car_data[driver_2]['block_start'] = practice_session.car_data[driver_2]['is_driving'] & (~practice_session.car_data[driver_2]['is_driving'].shift(1, fill_value=False))
        practice_session.car_data[driver_2]['block_id'] = practice_session.car_data[driver_2]['block_start'].cumsum()
        df_driving_only = practice_session.car_data[driver_2][practice_session.car_data[driver_2]['is_driving']].copy()

        if not df_driving_only.empty:
            fastest_lap = practice_session.laps.pick_drivers(driver_2).pick_fastest()
            driver_2 = driver_info_2["Abbreviation"]

            driving_block_durations = df_driving_only[df_driving_only['block_id'] > 0].groupby('block_id')['SessionTime'].apply(lambda x: x.iloc[-1] - x.iloc[0])
            min_block_duration = pd.Timedelta(minutes=1)
            filtered_driving_block_durations = driving_block_durations[driving_block_durations >= min_block_duration]
            
            total_run_time_driver_2 = str(filtered_driving_block_durations.sum())[10:-3]
            try:
                fastest_lap_driver_2 = str(fastest_lap['LapTime'])[10:-3]
            except:
                fastest_lap_driver_2 = 'No Data'

            driver_stints_driver_2 = stints[stints['Driver'] == driver_2]

            data_driver_2 = []
            try:
                for index, row in driver_stints_driver_2.iterrows():
                    driver_lap = practice_session.laps.pick_drivers(driver_2).pick_laps(range(int(row['FirstLap']), int(row['LastLap'])+1)).pick_quicklaps()
                    number_lap_driver_2 = int(driver_lap['LapTime'].count())
                    driver_avg_lap_driver_2 = str(timedelta(seconds = driver_lap['LapTime'].dt.total_seconds().median()))[3:-3]
                    compound_driver_2 = stints['Compound'][index]
                    data_driver_2.append(compound_driver_2)
                    data_driver_2.append(number_lap_driver_2)
                    data_driver_2.append(driver_avg_lap_driver_2)
            except:
                driver_avg_lap_driver_2 = 'No Average Lap Time Found'
                compound_driver_2 = 'No Compound Found'
                number_lap_driver_2 = 'No Lap completed'
            data_driver_2_len = int(len(data_driver_2)/3)

        race_name = str(practice_session)[20:]
        
        team_logo = f'/home/kurios/Documents/f1_analysis/data/external/team_logos/{team}.png'
        
        prs.slide_width = Pt(1080)
        prs.slide_height = Pt(1350)
        blank_slide_layout = prs.slide_layouts[5]
        slide = prs.slides.add_slide(blank_slide_layout)

        #BACKGROUND
        background = slide.background
        fill = background.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(21, 21, 30)

        #HEADER
        f1_logo = '/home/kurios/Documents/f1_analysis/data/external/team_logos/F1_75_Logo.png'
        pic = slide.shapes.add_picture(f1_logo, Pt(40), Pt(54), height=Pt(38), width= Pt(200))

        title = slide.shapes.title
        title.text = race_name
        title.top = Pt(84)
        title.left = Pt(215)
        title.width = Pt(600)

        title.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)
        title.text_frame.paragraphs[0].font.size = Pt(38)
        title.text_frame.paragraphs[0].font.name = 'Formula1 Display Bold'
         
        pic = slide.shapes.add_picture(team_logo, Pt(820), Pt(25), height= Pt(100), width=Pt(200))

        #STRUCTURE
        line1=slide.shapes.add_shape(MSO_CONNECTOR.STRAIGHT, Pt(40), Pt(150), Pt(1000), Pt(2))
        line1.line.fill.background()
        fill = line1.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(244, 244, 244)

        line2=slide.shapes.add_shape(MSO_CONNECTOR.STRAIGHT, Pt(40), Pt(250), Pt(1000), Pt(2))
        line2.line.fill.background()
        fill = line2.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(244, 244, 244)

        line4=slide.shapes.add_shape(MSO_CONNECTOR.STRAIGHT, Pt(539), Pt(250), Pt(2), Pt(1000))
        line4.line.fill.background()
        fill = line4.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(244, 244, 244)

        #REFERENCES
        txBox = slide.shapes.add_textbox(left=Pt(170), top= Pt(150), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Air Temp Range"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)
        
        txBox = slide.shapes.add_textbox(left=Pt(525), top= Pt(150), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Track Temp Range"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)

        txBox = slide.shapes.add_textbox(left=Pt(910), top= Pt(150), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Humidity Range"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)

        txBox = slide.shapes.add_textbox(left=Pt(70), top= Pt(250), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Driver"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)
        
        txBox = slide.shapes.add_textbox(left=Pt(1010), top= Pt(250), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Driver"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)

        txBox = slide.shapes.add_textbox(left=Pt(95), top= Pt(300), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Fastest Lap"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)
        
        txBox = slide.shapes.add_textbox(left=Pt(985), top= Pt(300), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Fastest Lap"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)

        txBox = slide.shapes.add_textbox(left=Pt(125), top= Pt(400), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Number Laps"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)

        txBox = slide.shapes.add_textbox(left=Pt(275), top= Pt(400), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Tyre Type"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)

        txBox = slide.shapes.add_textbox(left=Pt(425), top= Pt(400), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Average Lap Time"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)
        
        txBox = slide.shapes.add_textbox(left=Pt(665), top= Pt(400), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Number Laps"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)

        txBox = slide.shapes.add_textbox(left=Pt(815), top= Pt(400), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Tyre Type"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)

        txBox = slide.shapes.add_textbox(left=Pt(965), top= Pt(400), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = "Average Lap Time"
        run.font.name = 'Formula1 Display Bold'
        run.font.size = Pt(16)
        p.font.color.rgb = RGBColor(120, 120, 120)

        if data_driver_1_len != 0:
            shape = slide.shapes.add_table(data_driver_1_len, 3,left=Pt(40), top= Pt(450), width=Pt(460), height=Pt(800))
            table = shape.table
            for i in range(0, data_driver_1_len):
                cell = table.cell(i, 0)
                fill = cell.fill
                fill.solid()
               
                fill.fore_color.rgb = RGBColor(21, 21, 30)

                text_frame = cell.text_frame
                text_frame.clear()  

                p = text_frame.add_paragraph()
                run = p.add_run()
                y = i * 3
                run.text = f'{data_driver_1[y+1]}'
                
                run.font.name = 'Formula1 Display Regular'
                run.font.size = Pt(26)
                run.font.bold = True
                run.font.color.rgb = RGBColor(255, 255, 255)
                p.alignment = PP_ALIGN.CENTER
                
                cell.vertical_anchor = MSO_ANCHOR.MIDDLE
                cell = table.cell(i, 1)
                fill = cell.fill
                fill.solid()
               
                fill.fore_color.rgb = RGBColor(21, 21, 30)

                text_frame = cell.text_frame
                text_frame.clear()  

                p = text_frame.add_paragraph()
                run = p.add_run()
                run.text = f'{data_driver_1[y]}'
                
                run.font.name = 'Formula1 Display Regular'
                run.font.size = Pt(26)
                run.font.bold = True
                run.font.color.rgb = RGBColor(255, 255, 255)
                p.alignment = PP_ALIGN.CENTER
                cell.vertical_anchor = MSO_ANCHOR.MIDDLE
                
                cell = table.cell(i, 2)
                fill = cell.fill
                fill.solid()
               
                fill.fore_color.rgb = RGBColor(21, 21, 30)

                text_frame = cell.text_frame
                text_frame.clear()  

                p = text_frame.add_paragraph()
                run = p.add_run()
                run.text = f'{data_driver_1[y+2]}'
                
                run.font.name = 'Formula1 Display Regular'
                run.font.size = Pt(26)
                run.font.bold = True
                run.font.color.rgb = RGBColor(255, 255, 255)
                p.alignment = PP_ALIGN.CENTER
                cell.vertical_anchor = MSO_ANCHOR.MIDDLE


        if data_driver_2_len != 0:
            shape = slide.shapes.add_table(data_driver_2_len, 3,left=Pt(580), top= Pt(450), width=Pt(460), height=Pt(800))
            table = shape.table
            for i in range(0, data_driver_2_len):
                cell = table.cell(i, 0)
                fill = cell.fill
                fill.solid()
               
                fill.fore_color.rgb = RGBColor(21, 21, 30)

                text_frame = cell.text_frame
                text_frame.clear()  

                p = text_frame.add_paragraph()
                run = p.add_run()
                y = i * 3
                run.text = f'{data_driver_2[y+1]}'
                
                run.font.name = 'Formula1 Display Regular'
                run.font.size = Pt(26)
                run.font.bold = True
                run.font.color.rgb = RGBColor(255, 255, 255)
                p.alignment = PP_ALIGN.CENTER
                cell.vertical_anchor = MSO_ANCHOR.MIDDLE
                
                cell = table.cell(i, 1)
                fill = cell.fill
                fill.solid()
               
                fill.fore_color.rgb = RGBColor(21, 21, 30)

                text_frame = cell.text_frame
                text_frame.clear()  

                p = text_frame.add_paragraph()
                run = p.add_run()
                run.text = f'{data_driver_2[y]}'
                
                run.font.name = 'Formula1 Display Regular'
                run.font.size = Pt(26)
                run.font.bold = True
                run.font.color.rgb = RGBColor(255, 255, 255)
                p.alignment = PP_ALIGN.CENTER
                cell.vertical_anchor = MSO_ANCHOR.MIDDLE
                
                cell = table.cell(i, 2)
                fill = cell.fill
                fill.solid()
               
                fill.fore_color.rgb = RGBColor(21, 21, 30)

                text_frame = cell.text_frame
                text_frame.clear()  

                p = text_frame.add_paragraph()
                run = p.add_run()
                run.text = f'{data_driver_2[y+2]}'
                
                run.font.name = 'Formula1 Display Regular'
                run.font.size = Pt(26)
                run.font.bold = True
                run.font.color.rgb = RGBColor(255, 255, 255)
                p.alignment = PP_ALIGN.CENTER
                cell.vertical_anchor = MSO_ANCHOR.MIDDLE
    

        #TRACK VARIABLE
        txBox = slide.shapes.add_textbox(left=Pt(170), top=Pt(170), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        p.alignment = PP_ALIGN.LEFT
        run = p.add_run()
        run.text = air_temp_range
        run.font.name = 'Formula1 Display Regular'
        run.font.size = Pt(30)
        run.font.bold = True
        run.font.color.rgb = RGBColor(255, 255, 255)

        txBox = slide.shapes.add_textbox(left=Pt(525), top= Pt(170), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = track_temp_range
        run.font.name = 'Formula1 Display Regular'
        run.font.size = Pt(30)
        run.font.bold = True
        run.font.color.rgb = RGBColor(255, 255, 255)

        txBox = slide.shapes.add_textbox(left=Pt(910), top= Pt(170), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = humidity_range
        run.font.name = 'Formula1 Display Regular'
        run.font.size = Pt(30)
        run.font.bold = True
        run.font.color.rgb = RGBColor(255, 255, 255)

        #DRIVERS VARIABLE
        txBox = slide.shapes.add_textbox(left=Pt(310), top=Pt(240), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        p.alignment = PP_ALIGN.LEFT
        run = p.add_run()
        run.text = driver_1_name
        run.font.name = 'Formula1 Display Regular'
        run.font.size = Pt(30)
        run.font.bold = True
        run.font.color.rgb = RGBColor(255, 255, 255)

        txBox = slide.shapes.add_textbox(left=Pt(770), top=Pt(240), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        p.alignment = PP_ALIGN.LEFT
        run = p.add_run()
        run.text = driver_2_name
        run.font.name = 'Formula1 Display Regular'
        run.font.size = Pt(30)
        run.font.bold = True
        run.font.color.rgb = RGBColor(255, 255, 255)
    
        txBox = slide.shapes.add_textbox(left=Pt(310), top=Pt(290), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        p.alignment = PP_ALIGN.LEFT
        run = p.add_run()
        run.text = fastest_lap_driver_1
        run.font.name = 'Formula1 Display Regular'
        run.font.size = Pt(30)
        run.font.bold = True
        run.font.color.rgb = RGBColor(255, 255, 255)

        txBox = slide.shapes.add_textbox(left=Pt(770), top=Pt(290), width=Pt(0), height=Pt(20))
        tf = txBox.text_frame
        p = tf.add_paragraph()
        p.alignment = PP_ALIGN.LEFT
        run = p.add_run()
        run.text = fastest_lap_driver_2
        run.font.name = 'Formula1 Display Regular'
        run.font.size = Pt(30)
        run.font.bold = True
        run.font.color.rgb = RGBColor(255, 255, 255)

    prs.save(f'{race_number}_{practice_session}.pptx')

FileNotFoundError: [Errno 2] No such file or directory: '/home/kurios/Documents/f1_analysis/reports/test/'

In [15]:
for practice_session in (session_fp1, session_fp2, session_fp3):
    #for driver in practice_session.drivers:
        driver = '16'
        fastest_lap = practice_session.laps.pick_drivers(driver).pick_fastest()
        fastest_lap_tel = fastest_lap.get_car_data()
        driver = practice_session.get_driver(driver)["Abbreviation"]
        compound = fastest_lap['Compound']
        tyre_life = fastest_lap['TyreLife']
        full_throttle = round(len(np.where(fastest_lap_tel['Throttle'].values >= 90)[0])/len(fastest_lap_tel)*100)
        brake = round(len(np.where(fastest_lap_tel['Brake'] == True)[0])/len(fastest_lap_tel)*100)
        cornering = 100 - full_throttle - brake
        i1_trap_speed = int(fastest_lap['SpeedI1'])
        i2_trap_speed = int(fastest_lap['SpeedI2'])
        fl_trap_speed = int(fastest_lap['SpeedFL'])
        speed_trap_speed = int(fastest_lap['SpeedST'])
        compound = fastest_lap['Compound']

        print(driver)
        print(str(fastest_lap['LapTime'])[10:-3])
        print('Compound : ' + compound)
        print('Tyre Life : ' + str(tyre_life))
        print('Full Throttle : ' + str(full_throttle))
        print('Brake : ' + str(brake))
        print('Cornering : ' + str(cornering))
        print('I1 Trap Speed : ' + str(i1_trap_speed) + ' kp/h')
        print('I2 Trap Speed : ' + str(i2_trap_speed) + ' kp/h')
        print('FL Trap Speed : ' + str(fl_trap_speed) + ' kp/h')
        print('Speed Trap Speed : ' + str(speed_trap_speed) + ' kp/h')

LEC
01:17.077
Compound : SOFT
Tyre Life : 2.0
Full Throttle : 68
Brake : 17
Cornering : 15
I1 Trap Speed : 230 kp/h
I2 Trap Speed : 255 kp/h
FL Trap Speed : 285 kp/h
Speed Trap Speed : 291 kp/h
LEC
01:15.768
Compound : SOFT
Tyre Life : 6.0
Full Throttle : 70
Brake : 17
Cornering : 13
I1 Trap Speed : 230 kp/h
I2 Trap Speed : 261 kp/h
FL Trap Speed : 284 kp/h
Speed Trap Speed : 296 kp/h
LEC
01:15.451
Compound : SOFT
Tyre Life : 8.0
Full Throttle : 68
Brake : 17
Cornering : 15
I1 Trap Speed : 230 kp/h
I2 Trap Speed : 262 kp/h
FL Trap Speed : 286 kp/h
Speed Trap Speed : 297 kp/h


On average an F1 car consume 2.5L per lap about 1.7kg of fuel per lap,
10 kg will have a 0.15% impact on lap time,
Maximum 110kg fuel 

Engine mode can be estimate based on lap at speed trap 
Can increase top speed by 5–15 km/h, depending on deployment length and car setup
Using ERS can gain you between 0.4% and 0.7% depending on the speed of the circuit

In [6]:
print(fastest_lap_tel)

                       Date      RPM  Speed  nGear  Throttle  Brake  DRS  \
0   2025-03-15 02:16:16.072  11176.0  315.0      8     100.0  False   12   
1   2025-03-15 02:16:16.232  11201.0  315.0      8     100.0  False   12   
2   2025-03-15 02:16:16.632  11309.0  319.0      8     100.0  False   12   
3   2025-03-15 02:16:16.992  11319.0  320.0      8     100.0  False   12   
4   2025-03-15 02:16:17.152  11428.0  320.0      8     100.0  False   12   
..                      ...      ...    ...    ...       ...    ...  ...   
279 2025-03-15 02:17:31.112  10768.0  297.0      7     100.0  False   12   
280 2025-03-15 02:17:31.312  10737.0  301.0      8     100.0  False   12   
281 2025-03-15 02:17:31.672  10877.0  306.0      8     100.0  False   12   
282 2025-03-15 02:17:32.072  10980.0  311.0      8     100.0  False   12   
283 2025-03-15 02:17:32.472  11093.0  312.0      8     100.0  False   12   

    Source                   Time            SessionTime  
0      car 0 days 00:00:00.1

In [42]:
for driver in practice_session.drivers:
    print(driver)

1
4
5
6
7
10
12
14
16
18
22
23
27
30
31
44
55
63
81
87
